In [ ]:
import subprocess
import json
import os
import pickle
import traceback
import socket

import numpy as np
import torch


process = subprocess.Popen('docker container start mobilitydb', shell=True)

In [ ]:
def is_notebook() -> bool:
    try:
        shell = get_ipython().__class__.__name__
        if shell == 'ZMQInteractiveShell':
            # Jupyter notebook or qtconsole
            return True
        elif shell == 'TerminalInteractiveShell':
            # Terminal running IPython
            return False
        else:
            # Other type (?)
            return False
    except NameError:
        # Probably standard Python interpreter
        return False


if is_notebook():
    %cd ..
    from tqdm.notebook import tqdm
else:
    from tqdm import tqdm

In [ ]:
process.wait()

In [ ]:
from spatialyze.video_processor.camera_config import camera_config
from spatialyze.video_processor.payload import Payload
from spatialyze.video_processor.pipeline import Pipeline
from spatialyze.video_processor.video import Video

In [ ]:
# Stages
from spatialyze.video_processor.stages.in_view import InView, InViewOld

In [ ]:
NUSCENES_PROCESSED_DATA = "NUSCENES_PROCESSED_DATA"

In [ ]:
print(NUSCENES_PROCESSED_DATA in os.environ)
print(os.environ['NUSCENES_PROCESSED_DATA'])

In [ ]:
DATA_DIR = os.environ[NUSCENES_PROCESSED_DATA]
with open(os.path.join(DATA_DIR, "videos", "frames.pkl"), "rb") as f:
    videos = pickle.load(f)

In [ ]:
with open(os.path.join(DATA_DIR, 'cities.pkl'), 'rb') as f:
    cities = pickle.load(f)

In [ ]:
def run_benchmark(pipeline, filename, ignore_error=False):
    keep = {}
    failed_videos = []
    names = cities['boston-seaport'][:]
    filtered_videos = [(n, v) for n, v in videos.items() if n[6:10] in names]
    for i, (name, video) in tqdm(enumerate(filtered_videos), total=len(filtered_videos)):
        if name[6:10] not in names:
            continue

        try:
            video_filename = video['filename']
            if not video_filename.startswith('boston'): continue
            # if not video_filename.startswith('boston') or 'FRONT' not in name: continue

            frames = Video(
                os.path.join(DATA_DIR, "videos", video["filename"]),
                [camera_config(*f, 0) for f in video["frames"]],
            )

            output = pipeline.run(Payload(frames))
            keep[name] = str(output.keep)
        except Exception as e:
            if ignore_error:
                message = str(traceback.format_exc())
                failed_videos.append((name, message))
                print(e)
                print(message)
                print("------------------------------------------------------------------------------------")
                print()
                print()
            else:
                raise e
    with open(f"./outputs/test-inview--keep--{filename}.json", "w") as f:
        json.dump(keep, f, indent=2)

    with open(f"./outputs/test-inview--failed--{filename}.json", "w") as f:
        json.dump(failed_videos, f, indent=2)

    with open(f"./outputs/test-inview--perf--{filename}.json", "w") as f:
        performance = [
            {
                "stage": stage.classname(),
                "benchmark": stage.benchmarks,
            }
            for stage
            in pipeline.stages
        ]
        json.dump(performance, f, indent=2)

In [ ]:
type_filter = ['bicycle', 'motorcycle', 'car', 'truck', 'bus']

In [ ]:
p0 = Pipeline()
p0.add_filter(InViewOld(50, 'intersection'))

run_benchmark(p0, 'inview-old2', ignore_error=True)

In [ ]:
p1 = Pipeline()
p1.add_filter(InView(50, 'intersection'))

run_benchmark(p1, 'inview-new', ignore_error=True)

In [ ]:
p2 = Pipeline()
p2.add_filter(InViewOld(50, 'intersection', min_distance=True))

run_benchmark(p2, 'inview-old', ignore_error=True)